<a href="https://colab.research.google.com/github/Ater97/DeepLearningKeras/blob/master/CatsVsDogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import glob
import matplotlib.pyplot as plt
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessomg.image import ImageDataGenerator
# Supress warnig
import os 
os.environ['TF_CPP_MiN_LOG_LEVEL']='2'

#Get count of fiiles in this folder and all subfolders
def get_num_files(path):
  if(not os.path.exissts(path))
     return 0 
  return sum([len(fles) for r,d, files in os.walk(path)])
#get count of number of subfolders 
def get_num_subfolder(path):
  if not os.path.exists(path):
    return 0
  return sum([len(fles) for r,d, files in os.walk(path)])

#definde image generators 
def create_img_generator():
  return ImageDataGenerator(preprocessing_function = preprocess_input,
                           rotation_range = 30,
                           width_shift_range = 0.2,
                           height_ahift_range = 0.2,
                           shear_range = 0.2,
                           zoom_range = 0.2,
                           horizontal_flip = True) #fake more images 
#Main
Image_width, Image_height = 299, 299
Training_Epochs = 2
Batch_Size = 32
Number_FC_Neurons = 1024

#***********************************change for an OpenFileDialog
train_dir = './data/train' 
validate_dir = './data/validate'

num_train_samples = get_num_files(train_dir)
num_classes = get_num_subfolders(train_dir)
num_valdate_samples = get_num_files(validate_dir)

num_epoch = Training_Epochs
batch_size = Batch_Size

#Define data pre-processing
# Define image generators for training and testing
train_image_gen = create_img_generator()
test_image_gen = create_img_generator()



# Connect the image generator to a folder contains the source images 
# Training image generator
train_generator = tain_image_gen.flow_from_directory(
  train_dir, 
  target_size = (Image_width, Image_height),
  batch_size = batch_size,
  seed = 42) # set seed for reproducability

# Validation image generator
validation_generator = test_image_gen.flow_from_directory(
  validate_dir,
  target_size = (Image_width, Image_height),
  batch_size = batch_size,
  seed = 42) # set seed for reproducability

#Load the Inception V3 model and load it with its pre trained weights, excluding the final
# Fully connected layer
InceptionV3_base_model = InceptionV3(weights = 'imagenet', 
                                     include_top = False) #exclde final FC layer
print('Inception v3 base model without las FC loaded')

#Define the layers in the new classification prediction 
x = InceptionV3_base_model.output
x = GlobalAveragePooling2d()(x)
x = Dense(Number_FC_Neurons, activation='relu')(x) #new FC layerm random init
predictions = Dense(num_classes, activation='softmax')(x) #new softmax layer

#Define trainable model wich links input from the Inception V3 base model to the new classification prediction layers
model= Model(inputs = InceptionV3_base_model.input, outputs = predictions)

#print model structure diagram
print(model.summary())
#-------------------------------------------------------------------------------
# Option 1 basic transfer learning
print('\nPerforming Transfer Learning')

for layer in InceptionV3_base_model.layers:
  layer.trainable = False

# Define model compile for basic Transfer Learning
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

# Fit the transfer learnign model to the data from the generators. 
# By using generators we can ask to request sample image and the generators will pull images from  the the training or validation folders and alter them slightly
history_transfer_learning = model.fit_generator(
  train_generator,
  epochs = num-epoch,
  steps_per_epoch = num_train_samples // batch_size,
  validation_data = validation_generator,
  validation_steps = num_validate_samples // batch_size,
  class_weight ='auto')

# Save transfer learning model
model.save('inceptionv3-transfer-lerning.model')

